<img src="../assets/logos/Logo_operations.png" width=500></img>


## Welcome!
Welcome to the opperations group! you are responsible for deploy the code and models for your team and this guide will walk you through how to do you job. You will:

1. launch asset bucket
1. launch resources
1. Train and deploy models
1. register your team
1. retrain and redeploy models
1. clean up resources when the game is done

## Launch Asset Bucket 
Now we are going to launch a cloudformation template that creates and S3 "Asset" bucket to hold all the assets we need to launch the main cloudformatio template. All you need to do is run the following cell

In [ ]:
!npm run bootstrap

## Launch

Lastly, we will launch the main cloudformation template with all the infastructure we will need. The following cell will launch a cloudformation template that has:

- aws-sagemaker-build pipeline for the Layout Endpoint
- aws-sagemaker-build pipeline for the Shoot Endpoint
- an SNS Topic and SQS Queue to recieve errors from the Game Engine
- A IAM role that the Game Engine will assume inorder to call your endpoint
- An AWS glue crawler and database for the game data

In [ ]:
!npm run up

## Train and Deploy Initial Models

now that you have the infastructure up you need to train and deploy your models. This repository already has code to deploy very simple Layout and Shoot endoint. the following two cells wil train and deploy those models. you can also run these commands in parrallel by opening up a terminal window. 

In [ ]:
!cd .. && ./bin/startLayoutPipeline.js

In [ ]:
!cd .. && ./bin/startShootPipeline.js

## Register Team
Once your two endpoints and deployed and up you can now register your team and start playing. Run the following cell to get the information needed to register your team

In [4]:
!../bin/register_outputs.sh

(node:44376) UnhandledPromiseRejectionWarning: Unhandled promise rejection (rejection id: 1): ValidationError: Stack with id battleship-bootstrap-dev-0 does not exist
(node:44376) [DEP0018] DeprecationWarning: Unhandled promise rejections are deprecated. In the future, promise rejections that are not handled will terminate the Node.js process with a non-zero exit code.


## Share Data Bucket
there is a single s3 bucket that data will need to be uploaded to inorder to be used by endpoint teams. You can find this bucket by running:

In [ ]:
!echo "Data Bucket:"
!../templates/main/bin/output.js | jq '.DataBucket'

## retrain and redeploy models

- Whent the Offensive and Defensive groups have changes they will commit them to their branches. Merge their branches into Dev first to check for any conflicts, then merge into master. 
```shell
git checkout shoot # or layout if it was the layout team
git pull
git checkout dev
git merge shoot
git checkout master
git merge dev
```

- run the following to update the infastructure
```shell
npm run update
```

- to update your models you can kick off new pipeline jobs using the same startLayoutPipeline.js and startShootPipeline.js scripts you ran earlier.
```shell
./bin/startLayoutPipeline.js #for layout changes
./bin/startShootPipeline.js #for shoot changes
```



## Checking for Errors
One of the resources that the earlier cloudformation template creates is an SQS Queue that holds Errors from the games. You should check this queue to make sure all the endpoints are working correctly. It might also be smart to set up a __cloud watch alarm__ to alert you when ever messages come into that queue

## Cleaning up
When the day is over go to the cloudformation console and delete all the cloudformation templates inorder to stop being charged for the resources and delete an IAM users you created for the event.

1. run the following command to delete the deployment resources
```shell
npm run down
```
```shell
npm run down_bootstrap
```
2. go the cloudformation console to delete the notebook template
3. go to the IAM console and delete any IAM users you created for the day